In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import adjusted_rand_score as ars
from sklearn.mixture import BayesianGaussianMixture as GMM_sklearn
from vbnigmm import BayesianGaussianMixture as GMM_takekawa
from vbnigmm import BayesianFixedNIGMixture as NIGMM_fixed
from vbnigmm import BayesianNIGMixture as NIGMM_takekawa

In [ ]:
def make_data(d, normality, asymmetry, difficulty, population, normality_prec=5, difficulty_prec=5):
    difficulty_prec += d
    m = len(population)

    mu = st.multivariate_normal(np.zeros(d), np.eye(d)).rvs(size=m)
    cov = difficulty ** 2 * st.wishart(difficulty_prec, np.eye(d) / difficulty_prec).rvs(size=m)
    beta = asymmetry * np.array([st.multivariate_normal(np.zeros(d), c).rvs() for c in cov])
    lmd = st.invgauss(1 / normality_prec, 0, normality * normality_prec).rvs(size=m)

    z = np.concatenate([np.full(n, i) for i, n in enumerate(population)])
    y = st.invgauss(1/lmd[z], 0, lmd[z]).rvs()
    x = np.array([st.multivariate_normal(m, s).rvs() for m, s in zip(mu[z]+y[:,None]*beta[z], y[:,None,None] * cov[z])])
    return pd.DataFrame(dict(x=x[:, 0], y=x[:, 1], z=x[:, 2], cluster=z+1))

In [ ]:
data = make_data(d=3, normality=0.3, asymmetry=0.5, difficulty=0.2, population=[100] * 10)
sns.pairplot(data, hue='cluster', markers='.', palette='prism')
plt.show()

In [ ]:
x = np.array(data[['x','y','z']])
y_true = np.array(data['cluster'])

In [ ]:
model = GMM_sklearn(50, n_init=3)
model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = GMM_takekawa(progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_fixed(progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(concentration_prior_type='dd', normality_prior_type='gamma', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(normality_prior_type='gamma', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(concentration_prior_type='dd', progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)

In [ ]:
model = NIGMM_takekawa(init_n=20, progress=1, num_try=3)
%time model.fit(x)
y_est = model.predict(x)
ars(y_true, y_est)